## 1. 환경 설정

In [1]:
import os
import torch

# 환경변수 설정 (스크립트 실행 전 혹은 코드 초반에)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

## 2. 데이터셋 준비

In [2]:
import os
from tello_dataset import create_tello_dataset

DATASET_FILE = "tello_dataset.json"

if not os.path.exists(DATASET_FILE):
    print(f"{DATASET_FILE} 생성 중...")
    create_tello_dataset()
    print(f"{DATASET_FILE} 생성 완료!")
else:
    print(f"{DATASET_FILE}이 이미 존재합니다.")

tello_dataset.json이 이미 존재합니다.


## 3. 모델 파인튜닝 (fp16 사용)
이 단계는 **A100과 같은 강력한 GPU(CUDA 환경)**가 필요하며, 모델 크기에 따라 **상당한 시간**이 소요됩니다.

아래 셀에서는 파인튜닝할 모델을 선택하고, LoRa와 `fp16`을 사용하여 안정적으로 파인튜닝을 수행하는 함수를 정의합니다. 양자화는 사용하지 않습니다.

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


In [4]:
import torch
torch.cuda.empty_cache()


In [10]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer


def run_finetuning_fp16(model_name, dataset_path, output_dir):
    print(f"--- {model_name} 파인튜닝 시작 (fp16) ---")
    
    os.environ["CUDA_VISIBLE_DEVICES"] = "3"
    torch.cuda.set_device(0)  # 이제 0번은 실제 물리 GPU 3번 (CUDA_VISIBLE_DEVICES로 리맵핑 됨)

    device = torch.device("cuda:0")  # 0번이 리맵핑된 GPU 3번 의미
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        # device_map 대신 device_map="auto" 또는 None 으로 하거나, 직접 model.to(device) 사용
        device_map=None,
        trust_remote_code=True
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = get_peft_model(model, peft_config)
    model = model.to(device)  # cuda:0은 실제로 물리 GPU 3번임

    print(f"Model is on device: {next(model.parameters()).device}")
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        save_steps=50,
        logging_steps=50,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=True,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="constant"
    )

    dataset = load_dataset('json', data_files=dataset_path, split="train")

    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        dataset_text_field="instruction",
        max_seq_length=None,
        tokenizer=tokenizer,
        args=training_args,
        packing=False
    )

    trainer.train()

    final_checkpoint_dir = os.path.join(output_dir, "final_checkpoint")
    trainer.model.save_pretrained(final_checkpoint_dir)
    tokenizer.save_pretrained(final_checkpoint_dir)
    
    print(f"--- {model_name} 파인튜닝 완료 ---")
    print(f"모델이 {final_checkpoint_dir}에 저장되었습니다.")


In [11]:

import torch
print("CUDA visible devices:", os.environ["CUDA_VISIBLE_DEVICES"])
print("torch.cuda.device_count():", torch.cuda.device_count())
print("Current CUDA device:", torch.cuda.current_device())
print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA visible devices: 3
torch.cuda.device_count(): 1
Current CUDA device: 0
CUDA device name: NVIDIA A100 80GB PCIe


### 파인튜닝 실행

아래 셀에서 파인튜닝할 3개의 한국어 모델을 설정합니다. `MODELS_TO_TRAIN` 리스트에 원하는 모델의 Hugging Face ID를 추가하거나 제거하여 실험을 변형할 수 있습니다.

**주의:** 아래 셀을 실행하면 리스트에 있는 각 모델에 대한 파인튜닝이 순차적으로 진행됩니다. A100 GPU에서도 상당한 시간이 소요될 수 있습니다.

In [12]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"

In [13]:
!pip show transformers accelerate

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/a202520153/.local/lib/python3.8/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers, trl
---
Name: accelerate
Version: 1.0.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /home/a202520153/.local/lib/python3.8/site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft, trl


| 모델 이름                          | 매개변수 수     | 아키텍처     | 학습 데이터                          | 특징 및 용도                                               | 라이센스         |
|-----------------------------------|------------------|--------------|--------------------------------------|------------------------------------------------------------|------------------|
| `beomi/Yi-Ko-6B`                  | 약 6B            | LLaMA 기반   | 한국어 중심의 웹 코퍼스             | Beomi가 한국어 최적화를 위해 공개한 LLaMA 파생 모델       | Apache 2.0       |
| `yanolja/KoSOLAR-10.7B-v0.2`      | 약 10.7B         | GPT 기반     | 한국어/영어 혼합 대규모 텍스트      | RAG 최적화, KoQA/KoMRC 성능 우수, 대화용 LLM 최적화       | CC-BY-NC 4.0     |
| `EleutherAI/polyglot-ko-5.8b`     | 약 5.8B          | GPT-J 기반   | OSCAR, The Pile + 한국어 데이터     | GPT-J 구조 기반의 한국어 전용 모델                         | Apache 2.0       |


In [14]:
MODELS_TO_TRAIN = [
    "beomi/Yi-Ko-6B",
    "yanolja/KoSOLAR-10.7B-v0.2",
    "EleutherAI/polyglot-ko-5.8b"
]

for model_name in MODELS_TO_TRAIN:
    output_dir = f"./{model_name.replace('/', '_')}-finetuned-fp16"
    run_finetuning_fp16(model_name, DATASET_FILE, output_dir)


--- beomi/Yi-Ko-6B 파인튜닝 시작 (fp16) ---


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model is on device: cuda:0


/home/a202520153/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/a202520153/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/a202520153/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/a202520153/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:412: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. 

Step,Training Loss


...

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Model is on device: cuda:0


/home/a202520153/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/a202520153/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/a202520153/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/a202520153/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:412: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. 

Step,Training Loss


--- EleutherAI/polyglot-ko-5.8b 파인튜닝 완료 ---
모델이 ./EleutherAI_polyglot-ko-5.8b-finetuned-fp16/final_checkpoint에 저장되었습니다.


## 4. 추론 및 결과 비교

모든 모델의 파인튜닝이 완료되면, 아래 셀을 실행하여 동일한 쿼리에 대한 각 모델의 Tello 명령어 생성 결과를 비교할 수 있습니다.
추론 시에도 파인튜닝과 동일하게 `fp16`을 사용하여 모델을 로드합니다.

In [29]:
from peft import PeftModel
import pandas as pd

def load_inference_model_fp16(base_model_name, peft_model_path):
    """파인튜닝된 LoRa 모델을 fp16으로 로드합니다."""
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=torch.float16, # Use fp16
        device_map="auto",
    )
    
    model = PeftModel.from_pretrained(model, peft_model_path)
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    return model, tokenizer

def run_inference(model, tokenizer, query):
    """주어진 쿼리에 대해 모델 추론을 실행하여 Tello 명령어를 생성합니다."""
    
    prompt = (
        "당신은 Tello 드론 명령어 변환기입니다.\n"
        "LoRA로 학습된 데이터를 참고하여, 사용자가 입력한 한국어 명령어를\n"
        "가장 적절한 Tello 드론 제어 명령어 JSON 형식으로 변환해 주세요.\n"
        "출력은 오직 JSON 형식의 명령어만 포함해야 하며, 설명, 주석, 기타 텍스트는 절대 포함하지 마세요.\n\n"
        f"### 사용자 명령어:\n{query}\n"
        "### Tello 명령어 (JSON 형식):\n"
    )
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # token_type_ids가 있으면 제거
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']
        
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50, 
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # JSON 부분만 추출
    json_start = response.find("{")
    json_end = response.rfind("}") + 1
    if json_start != -1 and json_end != -1:
        json_only = response[json_start:json_end]
    else:
        json_only = response.strip()
    
    return json_only

### 추론 실행 및 결과 비교표 생성

아래 셀에 비교하고 싶은 쿼리를 입력하고 실행하세요. 파인튜닝된 각 모델이 생성한 Tello 명령어를 표 형태로 보여줍니다.

In [31]:
query = "이륙해서 50cm 위로 올라간 다음에, 시계방향으로 한 바퀴 돌아줘."
results = []

print(f"입력 쿼리: '{query}'")
print("-"*50)

for model_name in MODELS_TO_TRAIN:
    base_model = model_name
    peft_path = f"./{model_name.replace('/', '_')}-finetuned-fp16/final_checkpoint"
    
    if not os.path.exists(peft_path):
        print(f"{peft_path} 에서 파인튜닝된 모델을 찾을 수 없어 추론을 건너뜁니다.")
        results.append({"Model": base_model, "Generated Command": "N/A (Not Fine-tuned)"})
        continue

    try:
        print(f"{base_model} 모델 로딩 중...")
        model, tokenizer = load_inference_model_fp16(base_model, peft_path)
        
        print(f"{base_model} 모델로 추론 실행 중...")
        generated_command = run_inference(model, tokenizer, query)
        results.append({"Model": base_model, "Generated Command": generated_command})
        
        print(f"추론 완료: {generated_command}")
        print("-"*50)
        
        # 메모리 확보를 위해 모델과 토크나이저를 명시적으로 삭제
        del model
        del tokenizer
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"{base_model} 추론 중 오류 발생: {e}")
        results.append({"Model": base_model, "Generated Command": f"Error: {e}"})

# Display results in a pandas DataFrame
print("=== 최종 비교 결과 ===")
df = pd.DataFrame(results)
display(df)

입력 쿼리: '이륙하고 고도 50미터 유지해.'
--------------------------------------------------
beomi/Yi-Ko-6B 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

beomi/Yi-Ko-6B 모델로 추론 실행 중...
추론 완료: {"type": "takeoff", "altitude": 50}
--------------------------------------------------
yanolja/KoSOLAR-10.7B-v0.2 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

yanolja/KoSOLAR-10.7B-v0.2 모델로 추론 실행 중...
추론 완료: 
--------------------------------------------------
EleutherAI/polyglot-ko-5.8b 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

EleutherAI/polyglot-ko-5.8b 모델로 추론 실행 중...
추론 완료: 당신은 Tello 드론 명령어 변환기입니다.
LoRA로 학습된 데이터를 참고하여, 사용자가 입력한 한국어 명령어를
가장 적절한 Tello 드론 제어 명령어 JSON 형식으로 변환해 주세요.
출력은 오직 JSON 형식의 명령어만 포함해야 하며, 설명, 주석, 기타 텍스트는 절대 포함하지 마세요.

### 사용자 명령어:
이륙하고 고도 50미터 유지해.
### Tello 명령어 (JSON 형식):
### 사용자 명령어:다면은### Tello 명령어 (JSON 형식):다면은### 사용자 명령어:다면은### Tello 명령어 (JSON 형식):다면은##
--------------------------------------------------
=== 최종 비교 결과 ===


,Model,Generated Command
0,beomi/Yi-Ko-6B,"{""type"": ""takeoff"", ""altitude"": 50}"
1,yanolja/KoSOLAR-10.7B-v0.2,
2,EleutherAI/polyglot-ko-5.8b,"당신은 Tello 드론 명령어 변환기입니다.\nLoRA로 학습된 데이터를 참고하여,..."


In [32]:
queries = [
    "이륙하고 고도 50미터 유지해.",
    "전진 100미터 후 착륙해.",
    "왼쪽으로 30도 회전하고 20미터 이동해.",
    "고도 10미터로 유지하면서 5미터 후진해.",
]

results = []

for query in queries:
    print(f"\n입력 쿼리: '{query}'")
    print("-"*50)
    
    for model_name in MODELS_TO_TRAIN:
        base_model = model_name
        peft_path = f"./{model_name.replace('/', '_')}-finetuned-fp16/final_checkpoint"
        
        if not os.path.exists(peft_path):
            print(f"{peft_path} 에서 파인튜닝된 모델을 찾을 수 없어 추론을 건너뜁니다.")
            results.append({
                "Query": query,
                "Model": base_model,
                "Generated Command": "N/A (Not Fine-tuned)"
            })
            continue

        try:
            print(f"{base_model} 모델 로딩 중...")
            model, tokenizer = load_inference_model_fp16(base_model, peft_path)

            print(f"{base_model} 모델로 추론 실행 중...")
            generated_command = run_inference(model, tokenizer, query)
            results.append({
                "Query": query,
                "Model": base_model,
                "Generated Command": generated_command
            })

            print(f"추론 완료: {generated_command}")
            print("-"*50)

            del model
            del tokenizer
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"{base_model} 추론 중 오류 발생: {e}")
            results.append({
                "Query": query,
                "Model": base_model,
                "Generated Command": f"Error: {e}"
            })

# 결과 출력
print("=== 최종 비교 결과 ===")
df = pd.DataFrame(results)
display(df)



입력 쿼리: '이륙하고 고도 50미터 유지해.'
--------------------------------------------------
beomi/Yi-Ko-6B 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

beomi/Yi-Ko-6B 모델로 추론 실행 중...
추론 완료: {"type": "takeoff", "altitude": 50}
--------------------------------------------------
yanolja/KoSOLAR-10.7B-v0.2 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

yanolja/KoSOLAR-10.7B-v0.2 모델로 추론 실행 중...
추론 완료: 
--------------------------------------------------
EleutherAI/polyglot-ko-5.8b 모델 로딩 중...


...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

yanolja/KoSOLAR-10.7B-v0.2 모델로 추론 실행 중...
추론 완료: {
    "cmd": "fwd",
    "speed": 0.5,
    "distance": 5,
    "altitude": 10
}
--------------------------------------------------
EleutherAI/polyglot-ko-5.8b 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

EleutherAI/polyglot-ko-5.8b 모델로 추론 실행 중...
추론 완료: 당신은 Tello 드론 명령어 변환기입니다.
LoRA로 학습된 데이터를 참고하여, 사용자가 입력한 한국어 명령어를
가장 적절한 Tello 드론 제어 명령어 JSON 형식으로 변환해 주세요.
출력은 오직 JSON 형식의 명령어만 포함해야 하며, 설명, 주석, 기타 텍스트는 절대 포함하지 마세요.

### 사용자 명령어:
고도 10미터로 유지하면서 5미터 후진해.
### Tello 명령어 (JSON 형식):
### 사용자 명령어:다면은### Tello 명령어 (JSON 형식):다면은### 사용자 명령어:다면은### Tello 명령어 (JSON 형식):다면은##
--------------------------------------------------
=== 최종 비교 결과 ===


,Query,Model,Generated Command
0,이륙하고 고도 50미터 유지해.,beomi/Yi-Ko-6B,"{""type"": ""takeoff"", ""altitude"": 50}"
1,이륙하고 고도 50미터 유지해.,yanolja/KoSOLAR-10.7B-v0.2,
2,이륙하고 고도 50미터 유지해.,EleutherAI/polyglot-ko-5.8b,"당신은 Tello 드론 명령어 변환기입니다.\nLoRA로 학습된 데이터를 참고하여,..."
3,전진 100미터 후 착륙해.,beomi/Yi-Ko-6B,
4,전진 100미터 후 착륙해.,yanolja/KoSOLAR-10.7B-v0.2,"{\n ""cmd"": ""forward"",\n ""distance"": 100,..."
5,전진 100미터 후 착륙해.,EleutherAI/polyglot-ko-5.8b,"당신은 Tello 드론 명령어 변환기입니다.\nLoRA로 학습된 데이터를 참고하여,..."
6,왼쪽으로 30도 회전하고 20미터 이동해.,beomi/Yi-Ko-6B,"{\n""type"": ""move"",\n""x"": 30,\n""y"": 20\n}"
7,왼쪽으로 30도 회전하고 20미터 이동해.,yanolja/KoSOLAR-10.7B-v0.2,"{\n ""left"": 30,\n ""distance"": 20\n}"
8,왼쪽으로 30도 회전하고 20미터 이동해.,EleutherAI/polyglot-ko-5.8b,"당신은 Tello 드론 명령어 변환기입니다.\nLoRA로 학습된 데이터를 참고하여,..."
9,고도 10미터로 유지하면서 5미터 후진해.,beomi/Yi-Ko-6B,


In [33]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

print("=== 최종 비교 결과 ===")
df = pd.DataFrame(results)
display(df)


=== 최종 비교 결과 ===


,Query,Model,Generated Command
0,이륙하고 고도 50미터 유지해.,beomi/Yi-Ko-6B,"{""type"": ""takeoff"", ""altitude"": 50}"
1,이륙하고 고도 50미터 유지해.,yanolja/KoSOLAR-10.7B-v0.2,
2,이륙하고 고도 50미터 유지해.,EleutherAI/polyglot-ko-5.8b,"당신은 Tello 드론 명령어 변환기입니다.\nLoRA로 학습된 데이터를 참고하여, 사용자가 입력한 한국어 명령어를\n가장 적절한 Tello 드론 제어 명령어 JSON 형식으로 변환해 주세요.\n출력은 오직 JSON 형식의 명령어만 포함해야 하며, 설명, 주석, 기타 텍스트는 절대 포함하지 마세요.\n\n### 사용자 명령어:\n이륙하고 고도 50미터 유지해.\n### Tello 명령어 (JSON 형식):\n### 사용자 명령어:다면은### Tello 명령어 (JSON 형식):다면은### 사용자 명령어:다면은### Tello 명령어 (JSON 형식):다면은##"
3,전진 100미터 후 착륙해.,beomi/Yi-Ko-6B,
4,전진 100미터 후 착륙해.,yanolja/KoSOLAR-10.7B-v0.2,"{\n ""cmd"": ""forward"",\n ""distance"": 100,\n ""land"": true\n}"
5,전진 100미터 후 착륙해.,EleutherAI/polyglot-ko-5.8b,"당신은 Tello 드론 명령어 변환기입니다.\nLoRA로 학습된 데이터를 참고하여, 사용자가 입력한 한국어 명령어를\n가장 적절한 Tello 드론 제어 명령어 JSON 형식으로 변환해 주세요.\n출력은 오직 JSON 형식의 명령어만 포함해야 하며, 설명, 주석, 기타 텍스트는 절대 포함하지 마세요.\n\n### 사용자 명령어:\n전진 100미터 후 착륙해.\n### Tello 명령어 (JSON 형식):\n### 사용자 명령어: 불거질### Tello 명령어 (JSON 형식):다면은### 사용자 명령어: 불거질### Tello 명령어 (JSON 형식):다면은##"
6,왼쪽으로 30도 회전하고 20미터 이동해.,beomi/Yi-Ko-6B,"{\n""type"": ""move"",\n""x"": 30,\n""y"": 20\n}"
7,왼쪽으로 30도 회전하고 20미터 이동해.,yanolja/KoSOLAR-10.7B-v0.2,"{\n ""left"": 30,\n ""distance"": 20\n}"
8,왼쪽으로 30도 회전하고 20미터 이동해.,EleutherAI/polyglot-ko-5.8b,"당신은 Tello 드론 명령어 변환기입니다.\nLoRA로 학습된 데이터를 참고하여, 사용자가 입력한 한국어 명령어를\n가장 적절한 Tello 드론 제어 명령어 JSON 형식으로 변환해 주세요.\n출력은 오직 JSON 형식의 명령어만 포함해야 하며, 설명, 주석, 기타 텍스트는 절대 포함하지 마세요.\n\n### 사용자 명령어:\n왼쪽으로 30도 회전하고 20미터 이동해.\n### Tello 명령어 (JSON 형식):\n오른쪽으로 30도 회전하고 20미터 이동해.다면은### Tello 명령어 (JSON 형식):다면은왼쪽으로 30도 회전하고 20미터 이동해.릎### Tello 명령어 (J"
9,고도 10미터로 유지하면서 5미터 후진해.,beomi/Yi-Ko-6B,
